In [35]:
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
import ast
import math

#################################################################################################################################
# day_of_week : W-MON, W-TUE, W-WED, W-THU, W-FRI, W-SAT, W-SUN
#################################################################################################################################
def analysis_timebase_sales_amount(year, day_of_week):
    
    conn = connect(host='salest-master-server', port=21050)
    cur = conn.cursor()

    cur.execute('USE salest')
    
    start_date = "%s/01/01" % year
    end_date = "%s/12/31" % year
    
    if(day_of_week=='All'):
        target_date_idx = pd.date_range(start_date,end_date)
    else:
        target_date_idx = pd.date_range(start_date,end_date, freq=day_of_week)
        
    target_date_arr = target_date_idx.strftime('%Y-%m-%d')
    target_date_tuple = tuple(target_date_arr)

    cur.execute(
        """
        SELECT time_hour, CAST(SUM(sales_amount) as INTEGER) AS total_amount, 
        COUNT(sales_amount) as num_of_transaction,
        COUNT(DISTINCT year_month_day) as date_count
        FROM(
            SELECT SUBSTR(date_receipt_num,1,10) AS year_month_day,
            SUBSTR(tr_time,1,2) AS time_hour,
            sales_amount
            FROM ext_tr_receipt WHERE SUBSTR(date_receipt_num,1,10) IN %s
            """ % (target_date_tuple,) +
            """
        ) view_tr_total_amount_by_dayofweek
        GROUP BY time_hour ORDER BY time_hour ASC
        """
    )
    df_by_weekofday = as_pandas(cur)
    conn.close()
        
    def calc_average_amount(row):
        return row.total_amount / row.date_count

    df_by_weekofday['total_amount'] = df_by_weekofday.apply(calc_average_amount,axis=1)
    df_by_weekofday.set_index('time_hour',inplace=True)
    
    return df_by_weekofday.to_dict()


analysis_timebase_sales_amount('2014','W-MON')

{'date_count': {'10': 6,
  '11': 5,
  '12': 7,
  '13': 9,
  '14': 8,
  '15': 10,
  '16': 9,
  '17': 9,
  '18': 7,
  '19': 9,
  '20': 8,
  '21': 10,
  '22': 9,
  '23': 6},
 'num_of_transaction': {'10': 27,
  '11': 28,
  '12': 23,
  '13': 60,
  '14': 67,
  '15': 61,
  '16': 33,
  '17': 56,
  '18': 26,
  '19': 57,
  '20': 53,
  '21': 43,
  '22': 44,
  '23': 29},
 'total_amount': {'10': 15866,
  '11': 16820,
  '12': 11742,
  '13': 23077,
  '14': 26512,
  '15': 20920,
  '16': 11655,
  '17': 18822,
  '18': 14871,
  '19': 19855,
  '20': 27037,
  '21': 16350,
  '22': 20222,
  '23': 13350}}